In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
import math

#TODO Introducir el dataset con el que se vaya a trabajar
station = 'escudero'
df = pd.read_csv(f'./data/data_{station}/{station}.csv')
cols = [col for col in df.columns]

In [7]:
df.head()

,date,dew,humidity,prec,trace,pres,station,ts,dir,speed,VRB_wind
0,1980-01-01,1.6,97.0,0.0,1.0,998.5,950001.0,2.1,330.0,10.0,0.0
1,1981-01-01 03:00:00,1.0,100.0,NaN,NaN,994.6,950001.0,1.0,0.0,0.0,0.0
2,1981-01-01 06:00:00,0.9,100.0,0.0,0.0,994.2,950001.0,0.9,0.0,0.0,0.0
3,1981-01-01 09:00:00,0.6,100.0,NaN,NaN,993.6,950001.0,0.6,0.0,0.0,0.0
4,1981-01-01 12:00:00,1.1,98.0,0.0,0.0,992.7,950001.0,1.3,260.0,7.0,0.0


In [8]:

# Pasamos la columna 'date' a formato datetime

try:
    print('ok')
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
except ValueError:  # Capturamos la excepción correcta
    for i in range(len(df)):
        fecha = df.loc[i, 'date']  

        if len(fecha) == 10:  # Verificamos si la fecha no tiene hora
            fecha += ' 00:00:00'  # Añadimos '00:00:00'
        
            # Convertimos la cadena a formato datetime
            df.loc[i, 'date'] = pd.to_datetime(fecha)


df.info()

ok
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221508 entries, 0 to 221507
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      221508 non-null  object 
 1   dew       221034 non-null  float64
 2   humidity  219193 non-null  float64
 3   prec      62085 non-null   float64
 4   trace     62085 non-null   float64
 5   pres      186430 non-null  float64
 6   station   221502 non-null  float64
 7   ts        221377 non-null  float64
 8   dir       220073 non-null  float64
 9   speed     221298 non-null  float64
 10  VRB_wind  221299 non-null  float64
dtypes: float64(10), object(1)
memory usage: 18.6+ MB


In [13]:
df.head()


,date,dew,humidity,prec,trace,pres,station,ts,dir,speed,VRB_wind
0,1980-01-01 00:00:00,1.6,97.0,0.0,1.0,998.5,950001.0,2.1,330.0,10.0,0.0
1,1981-01-01 03:00:00,1.0,100.0,NaN,NaN,994.6,950001.0,1.0,0.0,0.0,0.0
2,1981-01-01 06:00:00,0.9,100.0,0.0,0.0,994.2,950001.0,0.9,0.0,0.0,0.0
3,1981-01-01 09:00:00,0.6,100.0,NaN,NaN,993.6,950001.0,0.6,0.0,0.0,0.0
4,1981-01-01 12:00:00,1.1,98.0,0.0,0.0,992.7,950001.0,1.3,260.0,7.0,0.0


In [12]:
df['date'] = pd.to_datetime(df['date'])


In [71]:

# Filtrar las filas con fechas inválidas
filas_invalidas = df[df['date'].isna()]

# # Mostrar las filas con fechas incorrectas
# print(filas_invalidas)

num_filas_invalidas = filas_invalidas.isnull().sum()
num_filas_invalidas


date        0
dew         0
hr          0
prec        0
trace       0
pres        0
station     0
skt         0
dir         0
vel         0
VRB_wind    0
dtype: int64

In [19]:
df = df[(df['date'].dt.year >= 2007) & (df['date'].dt.month >= 2) & (df['date'].dt.day <= 10)]


In [20]:
df.head()

,date,dew,humidity,prec,trace,pres,station,ts,dir,speed,VRB_wind
88726,2007-03-01 00:00:00,-0.4,93.0,0.0,0.0,991.6,950001.0,0.6,330.0,11.0,0.0
88727,2007-03-01 01:00:00,-0.7,93.0,NaN,NaN,NaN,950001.0,0.3,340.0,10.0,0.0
88728,2007-03-01 02:00:00,-0.6,93.0,NaN,NaN,NaN,950001.0,0.4,340.0,10.0,0.0
88729,2007-03-01 03:00:00,-0.6,91.0,NaN,NaN,991.2,950001.0,0.7,330.0,11.0,0.0
88730,2007-03-01 04:00:00,-2.4,79.0,NaN,NaN,NaN,950001.0,0.8,340.0,9.0,0.0


In [3]:

# En caso de que los nulos tengan un formato diferente

df.replace(444, np.nan, inplace=True)

### Calcular la humedad relativa

Se aplica la formula de magnus a partir de la temperatura y el punto de rocío.

In [14]:
# Definir la función para calcular la presión de vapor de saturación utilizando la ecuación de Magnus
def presion_vapor_saturacion(T):
    return 6.1094 * np.exp((17.625 * T) / (T + 243.04))

# Definir la función para calcular la humedad relativa
def calcular_humedad_relativa(temp_aire, temp_rocio):
    # Calcular la presión de vapor para la temperatura del aire y el punto de rocío
    e_aire = presion_vapor_saturacion(temp_aire)
    e_rocio = presion_vapor_saturacion(temp_rocio)
    
    # Calcular la humedad relativa
    humedad_relativa = 100 * (e_rocio / e_aire)
    
    return humedad_relativa

# Aplicar la función a tu DataFrame, asumiendo que 'temp' es la temperatura del aire y 'd2m' el punto de rocío
df['hr'] = calcular_humedad_relativa(df['temp'], df['d2m'])

# Mostrar las primeras filas con la nueva columna de humedad relativa
print(df[['temp', 'd2m', 'hr']].head(30))

        temp  d2m          hr
201207   1.9  1.9  100.000000
201206   1.5  1.5  100.000000
201205   2.0  2.0  100.000000
201204   1.6  1.6  100.000000
201203   5.5  2.9   83.343240
201202   5.4  2.0   78.722874
201201   3.7  1.5   85.540069
201200   3.6  1.6   86.763330
201199   3.4  1.5   87.367093
201198   3.5  1.6   87.376700
201197   3.5  1.6   87.376700
201196   3.8  1.6   85.550941
201195   3.5  1.5   86.753294
201194   3.7  1.5   85.540069
201193   3.1  1.6   89.878996
201192   2.7  1.9   94.465770
201191   2.8  1.6   91.808170
201190   3.4  1.5   87.367093
201189   3.7  1.4   84.929269
201188   2.4  1.8   95.813257
201187   2.2  1.9   97.883395
201186   2.5  2.0   96.502879
201185   2.7  2.1   95.823269
201184   2.7  1.6   92.461451
201183   2.2  1.5   95.123011
201182   1.0  1.0  100.000000
201181   1.2  1.2  100.000000
201180   0.8  0.8  100.000000
201179   1.3  1.3  100.000000
201178   0.5  0.5  100.000000


In [21]:
cols

['date',
 'dew',
 'humidity',
 'prec',
 'trace',
 'pres',
 'station',
 'ts',
 'dir',
 'speed',
 'VRB_wind']

### Uniformamos el dataset

-Eliminamos las columnas que no nos interesen

-Fijamos la columan de fecha

In [22]:
# date,temp,skt,dir,vel,hr,prec,pres

columns_to_drop = [
 'dew',
 'trace',
  'station', 'VRB_wind'
    ]

df_2 = df.drop(columns=columns_to_drop)

df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35884 entries, 88726 to 221507
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      35884 non-null  datetime64[ns]
 1   humidity  35375 non-null  float64       
 2   prec      6980 non-null   float64       
 3   pres      31343 non-null  float64       
 4   ts        35844 non-null  float64       
 5   dir       35551 non-null  float64       
 6   speed     35848 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.2 MB


In [23]:
df_2['date'] = df_2['date'].apply(lambda x: x if ':' in str(x) else str(x) + ' 00:00:00')
df_2['date'].head()

df_2['date'] = pd.to_datetime(df_2['date'])

In [24]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35884 entries, 88726 to 221507
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      35884 non-null  datetime64[ns]
 1   humidity  35375 non-null  float64       
 2   prec      6980 non-null   float64       
 3   pres      31343 non-null  float64       
 4   ts        35844 non-null  float64       
 5   dir       35551 non-null  float64       
 6   speed     35848 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.2 MB


In [25]:
#Recuento de nulos

df_2.head()

,date,humidity,prec,pres,ts,dir,speed
88726,2007-03-01 00:00:00,93.0,0.0,991.6,0.6,330.0,11.0
88727,2007-03-01 01:00:00,93.0,NaN,NaN,0.3,340.0,10.0
88728,2007-03-01 02:00:00,93.0,NaN,NaN,0.4,340.0,10.0
88729,2007-03-01 03:00:00,91.0,NaN,991.2,0.7,330.0,11.0
88730,2007-03-01 04:00:00,79.0,NaN,NaN,0.8,340.0,9.0


In [18]:
# Recuento de fechas duplicadas
duplicates = df_2.duplicated(subset='date', keep='first').sum()

duplicates

0

### Reindexación de las columnas

In [51]:
cols = [col for col in df.columns]
cols

['date', 'hr', 'prec', 'pres', 'skt', 'dir', 'vel']

In [52]:
# date,temp,skt,dir,vel,hr,prec,pres
new_order = ['date', 'skt', 'dir', 'vel', 'hr', 'prec' , 'pres']
df = df.reindex(columns=new_order)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35884 entries, 0 to 35883
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    35884 non-null  datetime64[ns]
 1   skt     35844 non-null  float64       
 2   dir     35551 non-null  float64       
 3   vel     35848 non-null  float64       
 4   hr      35375 non-null  float64       
 5   prec    6980 non-null   float64       
 6   pres    31343 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.9 MB


In [57]:
df.to_csv(f'./data/data_{station}/{station}_clean.csv', index=False, columns=new_order)

# Visualizacion de los datos

In [81]:
station = 'gdc'

In [82]:
df = pd.read_csv(f'./data/data_{station}/{station}_clean.csv')

In [9]:

num_cols = len(df_2.columns) - 1  
num_rows = math.ceil(num_cols / 4)  


fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, 5 * num_rows))  

# Aplanar el array de ejes para poder iterar sobre él
axes = axes.flatten()

# Iterar sobre las columnas (excluyendo 'fhora') y generar gráficos
for ax, col in zip(axes, df_2.columns.drop('date')):
    print(col, '--->' ,type(df_2[col]))
    df_2[col] = pd.to_numeric(df_2[col], errors='coerce')
    ax.plot(df_2['date'], df_2[col])
    ax.set_xlabel('date')
    ax.set_ylabel(col)
    ax.set_title(f'{col} a lo largo del tiempo')

# Eliminar los ejes sobrantes si el número de columnas no es múltiplo de 4
if num_cols % 4:
    for ax in axes[num_cols:]:
        ax.remove()

plt.savefig(f'resource/clean_{station}.png')

plt.tight_layout()
plt.show()

skt ---> <class 'pandas.core.series.Series'>
dir ---> <class 'pandas.core.series.Series'>
vel ---> <class 'pandas.core.series.Series'>
hr ---> <class 'pandas.core.series.Series'>
prec ---> <class 'pandas.core.series.Series'>
pres ---> <class 'pandas.core.series.Series'>


KeyboardInterrupt: 